## Google Agent Development Kit

Today I'll be studying Google's latest announcement related to agents.
Here is the [announcement post](https://developers.googleblog.com/en/agent-development-kit-easy-to-build-multi-agent-applications/).
Here are the [docs](https://google.github.io/adk-docs/).

My goals are to:
1. Understand the core mental model of this Google project in regards to agents
2. Run the basic examples in a notebook
3. Read the API docs
4. Generate new research questions

## API Keys

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Getting started

Copy-pasted example from Google ADK docs, validating set up.

In [2]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.adk.models.lite_llm import LiteLlm

from IPython.display import display, Markdown

import sys
sys.path.append("..")
from common_utils import display_signature


Similar to OpenAI's Agents SDK, the core objects is `Agent`.

In [3]:
APP_NAME = "question_answer_agent"
USER_ID = "user_123"
SESSION_ID = "session_123"

In [4]:
agent = Agent(
    model="gemini-2.0-flash-exp",
    name="question_answer_agent",
    description="""an agent whose job it is to perform Google search queries and answer questions about the results.""",
    instruction="""You are an agent whose job is to perform Google search queries and answer questions about the results.""",
    tools=[google_search],
)

In [5]:
session_service = InMemorySessionService()

In [6]:
# session = session_service.create_session(
#     app_name=APP_NAME,
#     user_id=USER_ID,
#     session_id=SESSION_ID
# )

print(f"Attempting to create session with ID: {SESSION_ID}")
await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session {SESSION_ID} created (or attempted to create).")

Attempting to create session with ID: session_123
Session session_123 created (or attempted to create).


In [7]:
runner = Runner(
    agent=agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)

In [8]:
from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
    """Sends a query to the agent and prints the final response."""
    print(f"\n>>> User Query: {query}")

    # Prepare the user's message in ADK format
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "Agent did not produce a final response." # Default

    # Key Concept: run_async executes the agent logic and yields Events.
    # We iterate through events to find the final answer.
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        # You can uncomment the line below to see *all* events during execution
        print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

        # Key Concept: is_final_response() marks the concluding message for the turn.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Assuming text response in the first part
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate: # Handle potential errors/escalations
                final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
            # Add more checks here if needed (e.g., specific error codes)
            break # Stop processing events once the final response is found

    print(f"<<< Agent Response: {final_response_text}")

In [9]:
await call_agent_async("What is the weather like in London?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)


>>> User Query: What is the weather like in London?
  [Event] Author: question_answer_agent, Type: Event, Final: True, Content: parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text="The weather in London, United Kingdom is partly cloudy with a 45% chance of rain. The temperature is 14°C (58°F), but it feels like 13°C (56°F). The humidity is around 90%.\n\nHere's the forecast for the next few days:\n\n*   **Monday, May 26:** Cloudy during the day and light rain at night. The temperature will be between 12°C (54°F) and 17°C (62°F).\n*   **Tuesday, May 27:** Light rain during the day and rain showers at night. The temperature will be between 13°C (56°F) and 17°C (63°F).\n*   **Wednesday, May 28:** Light rain during the day and mostly cloudy at night. The temperature will be between 12°C (54°F) and 18°C (65°F).")] role='model'
<<< Agent Response: The weather in Lo

In [10]:
query = "What is the weather like in London?"
content = types.Content(role='user', parts=[types.Part(text=query)])

events = []
for event in runner.run(
    user_id=USER_ID,
    session_id=SESSION_ID, 
    new_message=content):
    events.append(event)

In [11]:
display(Markdown((events[-1].content.parts[0].text)))

The weather in London, United Kingdom is partly cloudy with a 45% chance of rain. The temperature is 14°C (58°F), but it feels like 13°C (56°F). The humidity is around 90%.

Here is the forecast for the next few days:

*   **Monday, May 26:** Cloudy during the day and light rain at night. The temperature will be between 12°C (54°F) and 17°C (62°F).
*   **Tuesday, May 27:** Light rain during the day and rain showers at night. The temperature will be between 13°C (56°F) and 17°C (63°F).
*   **Wednesday, May 28:** Light rain during the day and mostly cloudy at night. The temperature will be between 12°C (54°F) and 18°C (65°F).


## Notes on [the samples](https://github.com/google/adk-python/tree/main/contributing/samples)

Similar abstraction, basically a direct competitor to OpenAI Agents SDK. Commonalities include:
- Agent runtime
- Multi-model via LiteLLM
- MCP integration

## What is an Agent?

> A self-contained execution unit designed to act autonomously to achieve specific goals. Agents can perform tasks, interact with users, utilize external tools, and coordinate with other agents.

### What does "interact with users" mean?

### What does "utilize external tools" mean? 

### How does multi-agent coordination function in relation to OpenAI Agents SDK? 


In [12]:
display_signature(Agent)

### LlmAgent

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**name**: <class 'str'> *

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**description**: <class 'str'> = 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**parent_agent**: typing.Optional[google.adk.agents.base_agent.BaseAgent] = None

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**sub_agents**: list[google.adk.agents.base_agent.BaseAgent] = <factory>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**before_agent_callback**: typing.Union[typing.Callable[[google.adk.agents.callback_context.CallbackContext], typing.Union[typing.Awaitable[typing.Optional[google.genai.types.Content]], google.genai.types.Content, NoneType]], list[typing.Callable[[google.adk.agents.callback_context.CallbackContext], typing.Union[typing.Awaitable[typing.Optional[google.genai.types.Content]], google.genai.types.Content, NoneType]]], NoneType] = None

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**after_agent_callback**: typing.Union[typing.Callable[[google.adk.agents.callback_context.CallbackContext], typing.Union[typing.Awaitable[typing.Optional[google.genai.types.Content]], google.genai.types.Content, NoneType]], list[typing.Callable[[google.adk.agents.callback_context.CallbackContext], typing.Union[typing.Awaitable[typing.Optional[google.genai.types.Content]], google.genai.types.Content, NoneType]]], NoneType] = None

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**model**: typing.Union[str, google.adk.models.base_llm.BaseLlm] = 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**instruction**: typing.Union[str, typing.Callable[[google.adk.agents.readonly_context.ReadonlyContext], typing.Union[str, typing.Awaitable[str]]]] = 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**global_instruction**: typing.Union[str, typing.Callable[[google.adk.agents.readonly_context.ReadonlyContext], typing.Union[str, typing.Awaitable[str]]]] = 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**tools**: list[typing.Union[typing.Callable, google.adk.tools.base_tool.BaseTool, google.adk.tools.base_toolset.BaseToolset]] = <factory>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**generate_content_config**: typing.Optional[google.genai.types.GenerateContentConfig] = None

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**disallow_transfer_to_parent**: <class 'bool'> = False

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**disallow_transfer_to_peers**: <class 'bool'> = False

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**include_contents**: typing.Literal['default', 'none'] = default

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**input_schema**: typing.Optional[type[pydantic.main.BaseModel]] = None

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**output_schema**: typing.Optional[type[pydantic.main.BaseModel]] = None

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**output_key**: typing.Optional[str] = None

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**planner**: typing.Optional[google.adk.planners.base_planner.BasePlanner] = None

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**code_executor**: typing.Optional[google.adk.code_executors.base_code_executor.BaseCodeExecutor] = None

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**examples**: typing.Union[list[google.adk.examples.example.Example], google.adk.examples.base_example_provider.BaseExampleProvider, NoneType] = None

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**before_model_callback**: typing.Union[typing.Callable[[google.adk.agents.callback_context.CallbackContext, google.adk.models.llm_request.LlmRequest], typing.Union[typing.Awaitable[typing.Optional[google.adk.models.llm_response.LlmResponse]], google.adk.models.llm_response.LlmResponse, NoneType]], list[typing.Callable[[google.adk.agents.callback_context.CallbackContext, google.adk.models.llm_request.LlmRequest], typing.Union[typing.Awaitable[typing.Optional[google.adk.models.llm_response.LlmResponse]], google.adk.models.llm_response.LlmResponse, NoneType]]], NoneType] = None

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**after_model_callback**: typing.Union[typing.Callable[[google.adk.agents.callback_context.CallbackContext, google.adk.models.llm_response.LlmResponse], typing.Union[typing.Awaitable[typing.Optional[google.adk.models.llm_response.LlmResponse]], google.adk.models.llm_response.LlmResponse, NoneType]], list[typing.Callable[[google.adk.agents.callback_context.CallbackContext, google.adk.models.llm_response.LlmResponse], typing.Union[typing.Awaitable[typing.Optional[google.adk.models.llm_response.LlmResponse]], google.adk.models.llm_response.LlmResponse, NoneType]]], NoneType] = None

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**before_tool_callback**: typing.Union[typing.Callable[[google.adk.tools.base_tool.BaseTool, dict[str, typing.Any], google.adk.tools.tool_context.ToolContext], typing.Union[typing.Awaitable[typing.Optional[dict]], dict, NoneType]], list[typing.Callable[[google.adk.tools.base_tool.BaseTool, dict[str, typing.Any], google.adk.tools.tool_context.ToolContext], typing.Union[typing.Awaitable[typing.Optional[dict]], dict, NoneType]]], NoneType] = None

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**after_tool_callback**: typing.Union[typing.Callable[[google.adk.tools.base_tool.BaseTool, dict[str, typing.Any], google.adk.tools.tool_context.ToolContext, dict], typing.Union[typing.Awaitable[typing.Optional[dict]], dict, NoneType]], list[typing.Callable[[google.adk.tools.base_tool.BaseTool, dict[str, typing.Any], google.adk.tools.tool_context.ToolContext, dict], typing.Union[typing.Awaitable[typing.Optional[dict]], dict, NoneType]]], NoneType] = None



In [13]:
display_signature(Runner)

### Runner

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**app_name**: str *

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**agent**: BaseAgent *

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**artifact_service**: Optional[BaseArtifactService] = None

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**session_service**: BaseSessionService *

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**memory_service**: Optional[BaseMemoryService] = None



In [14]:
display_signature(Runner.run)

### run

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**self** *

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**user_id**: str *

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**session_id**: str *

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**new_message**: types.Content *

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**run_config**: RunConfig = speech_config=None response_modalities=None save_input_blobs_as_artifacts=False support_cfc=False streaming_mode=<StreamingMode.NONE: None> output_audio_transcription=None input_audio_transcription=None max_llm_calls=500



## Monitoring

### [Events](https://google.github.io/adk-docs/events/)

An immutable record representing a specific point in the agent's execution.
Let's run the example again and inspect the event object.

Google ADK also provides a GUI for interacting with agents, and viewing traces and events.

![](../static/google-adk-web-trace.png)

In [15]:
query = "What is the weather like in London?"
content = types.Content(role='user', parts=[types.Part(text=query)])

for event in runner.run(
    user_id=USER_ID,
    session_id=SESSION_ID, 
    new_message=content):
    print(event)

content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='The weather in London, United Kingdom is partly cloudy with a 45% chance of rain. The temperature is 14°C (58°F), but it feels like 13°C (56°F). The humidity is around 90%.\n\nHere is the forecast for the next few days:\n\n*   **Monday, May 26:** Cloudy during the day and light rain at night. The temperature will be between 12°C (54°F) and 17°C (62°F).\n*   **Tuesday, May 27:** Light rain during the day and rain showers at night. The temperature will be between 13°C (56°F) and 17°C (63°F).\n*   **Wednesday, May 28:** Light rain during the day and mostly cloudy at night. The temperature will be between 12°C (54°F) and 18°C (65°F).\n')], role='model') grounding_metadata=GroundingMetadata(grounding_chunks=[GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain=None, title='Weather i

In [ ]:
s = f"### Author: {event.author}"
s += f"\nId: {event.id}"
s += f"\n\nInvocation ID: {event.invocation_id}"
s += f"\n#### Content"
s += f"\n{event.content}"
s += f"\n#### Thought"
s += f"\n{event.content.parts[0].thought}"
s += f"\n#### Code Execution Result"
s += f"\n{event.content.parts[0].code_execution_result}"
s += f"\n#### Executable Code"
s += f"\n{event.content.parts[0].executable_code}"
s += f"\n#### File Data"
s += f"\n{event.content.parts[0].file_data}"
s += f"\n#### Function Call"
s += f"\n{event.content.parts[0].function_call}"
s += f"\n#### Function Response"
s += f"\n{event.content.parts[0].function_response}"

display(Markdown(s))

### TODO: Export traces

- https://weave-docs.wandb.ai/guides/integrations/google_adk/
- https://googleapis.github.io/genai-toolbox/how-to/export_telemetry/

## TODO: Evaluation

## TODO: Custom Models